In [2]:
# pip install py2neo

In [13]:
# pip install tqdm

In [14]:
from py2neo import Graph
import pandas as pd
from tqdm.auto import tqdm

In [6]:
graph = Graph("bolt://neo4j:7687", auth=("neo4j", "qwer1234"))

In [8]:
df_board = pd.read_csv('data/company-board.csv')
df_board

,name,positions,company
0,นาย ครรชิต สิงห์สุวรรณ์,"ประธานกรรมการ,กรรมการอิสระ,กรรมการตรวจสอบ",2s
1,นาย สมบัติ ลีสวัสดิ์ตระกูล,"ประธานกรรมการบริหาร,กรรมการ",2s
2,นาย สุรพล สติมานนท์,"รองประธานกรรมการ,กรรมการอิสระ,ประธานกรรมการตรว...",2s
3,นาย แสงรุ้ง นิติภาวะชน,"กรรมการผู้จัดการ,กรรมการ",2s
4,น.ส. บุญศรี ปัญญาเปี่ยมศักดิ์,กรรมการ,2s
...,...,...,...
7189,นาย ศุภกิจ งามจิตรเจริญ,"ประธานกรรมการบริหาร,ประธานเจ้าหน้าที่บริหาร,กร...",ziga
7190,น.ส. วราลักษณ์ งามจิตรเจริญ,"กรรมการผู้จัดการ,กรรมการ",ziga
7191,นาง มณฑา ทัสฐาน,กรรมการ,ziga
7192,นาย พลรชฏ เปียถนอม,"กรรมการอิสระ,กรรมการตรวจสอบ",ziga


In [9]:
df_share_holder = pd.read_csv('data/company-shareholder.csv')
df_share_holder

,sequence,name,nationality,numberOfShare,percentOfShare,isThaiNVDR,company
0,1,นาย สมบัติ ลีสวัสดิ์ตระกูล,THAI,95234050,19.05,False,2s
1,2,นาย แสงรุ้ง นิติภาวะชน,THAI,29109087,5.82,False,2s
2,3,นาย ปรีดา วงศ์สถิตย์พร,THAI,22397165,4.48,False,2s
3,4,น.ส. ธนิกา ตั้งพูนผลวิวัฒน์,THAI,20048000,4.01,False,2s
4,5,น.ส. นิสาภ์ รุ่งรัตนาอุบล,THAI,17962500,3.59,False,2s
...,...,...,...,...,...,...,...
8194,6,นาย ณัฐพล ทรงสายชลชัย,THAI,11201000,1.83,False,ziga
8195,7,นาง เล็ก ตรีมุทธาพงศ์,THAI,11091100,1.81,False,ziga
8196,8,นาง ลีนา วนวิทย์,THAI,6784900,1.11,False,ziga
8197,9,นาง จันทิรา ลือสกุล,THAI,6751001,1.10,False,ziga


In [10]:
_board_data = []
for index, item in df_board.iterrows():
    _board_data += [{
        'name': item.get('name', ""),
        'position': elm,
        'company': item.get('company', "")
    } for elm in item.get('positions', '').split(',')]
df_board2 = pd.DataFrame(_board_data)
df_board2

,name,position,company
0,นาย ครรชิต สิงห์สุวรรณ์,ประธานกรรมการ,2s
1,นาย ครรชิต สิงห์สุวรรณ์,กรรมการอิสระ,2s
2,นาย ครรชิต สิงห์สุวรรณ์,กรรมการตรวจสอบ,2s
3,นาย สมบัติ ลีสวัสดิ์ตระกูล,ประธานกรรมการบริหาร,2s
4,นาย สมบัติ ลีสวัสดิ์ตระกูล,กรรมการ,2s
...,...,...,...
11028,นาง มณฑา ทัสฐาน,กรรมการ,ziga
11029,นาย พลรชฏ เปียถนอม,กรรมการอิสระ,ziga
11030,นาย พลรชฏ เปียถนอม,กรรมการตรวจสอบ,ziga
11031,นาง เอมอร โชติยะปุตตะ,กรรมการอิสระ,ziga


In [21]:
tx = graph.begin()
for company in tqdm(df_board.company.unique()):
    tx.run("CREATE (c:Company {name:$name})", name=company)
tx.commit()

  0%|          | 0/746 [00:00<?, ?it/s]

/tmp/ipykernel_12/2053918837.py:4: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


In [22]:
df_person = pd.concat([
    df_board[['name']],
    df_share_holder[['name']]
])
df_person

,name
0,นาย ครรชิต สิงห์สุวรรณ์
1,นาย สมบัติ ลีสวัสดิ์ตระกูล
2,นาย สุรพล สติมานนท์
3,นาย แสงรุ้ง นิติภาวะชน
4,น.ส. บุญศรี ปัญญาเปี่ยมศักดิ์
...,...
8194,นาย ณัฐพล ทรงสายชลชัย
8195,นาง เล็ก ตรีมุทธาพงศ์
8196,นาง ลีนา วนวิทย์
8197,นาง จันทิรา ลือสกุล


In [30]:
tx = graph.begin()
for person in tqdm(df_person.name.unique()):
    tx.run("CREATE (p:Person {name:$name})", name=person)
graph.commit(tx)

  0%|          | 0/9787 [00:00<?, ?it/s]

TypeError: commit() missing 1 required positional argument: 'tx'

In [39]:
tx = graph.begin()
for index, row in tqdm(df_board2.iterrows(), total=df_board2.shape[0]):
    tx.run("""
        MATCH (p: Person), (c: Company)
        WHERE p.name = $person AND c.name = $company
        CREATE (p)-[r:RELTYPE {name: $relation}]->(c)
    """, person=row.get('name',''), company=row.get('company',""), relation=row.get('position',""))
    if index%1000 == 0:
        graph.commit(tx)
        tx = graph.begin()
graph.commit(tx)

  0%|          | 0/11033 [00:00<?, ?it/s]

In [40]:
tx = graph.begin()
for index, row in tqdm(df_share_holder.iterrows(), total=df_share_holder.shape[0]):
    tx.run("""
        MATCH (p: Person), (c: Company)
        WHERE p.name = $person AND c.name = $company
        CREATE (p)-[r:RELTYPE {name: $relation}]->(c)
    """, person=row.get('name',''), company=row.get('company',""), relation="share_holder")
    if index%1000 == 0:
        graph.commit(tx)
        tx = graph.begin()
graph.commit(tx)

  0%|          | 0/8199 [00:00<?, ?it/s]